## **PS4E9: Auto EDA & Modeling**
This notebook contains Auto EDA and Auto ML approaches for the regression of used car prices as part of the playground series, season 4, episode 9. In detail, it shows:

* Loading of all relevant datasets and first quick analysis of them
* Auto EDA through ydata_profiling
* Feature engineering & preprocessing
* Analysis of most important features
* AutoML model training with AutoGluon
* Submitting predictions using the best model based on the validation

**References:**
* The competition page: https://www.kaggle.com/competitions/playground-series-s4e9/
* The original dataset: https://www.kaggle.com/datasets/taeefnajib/used-car-price-prediction-dataset/data

## **Importing/Installing Libraries**

In [ ]:
!pip install -q autogluon.tabular

In [ ]:
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
from sklearn.preprocessing import TargetEncoder
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor, plot_tree
from autogluon.tabular import TabularPredictor
import re

## **Understanding the data**

In [ ]:
df_train = pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv', index_col='id')
df_test = pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv', index_col='id')
df_original = pd.read_csv('/kaggle/input/used-car-price-prediction-dataset/used_cars.csv')
df_sub = pd.read_csv('/kaggle/input/playground-series-s4e9/sample_submission.csv')

We first refine two columns of the original dataset so that we can use the data in our training:

In [ ]:
# thanks to https://www.kaggle.com/competitions/playground-series-s4e9/discussion/531333#2975520 for this code snippet
df_original[['milage', 'price']] = df_original[['milage', 'price']].map(
    lambda x: int(''.join(re.findall(r'\d+', x))))

Now, we can merge the training data of the competition and the orginal dataset to one big training set:

In [ ]:
df_train = pd.concat([df_train, df_original]).reset_index(drop=True)

Let's get some information of this training set:

In [ ]:
df_train.head()

In [ ]:
df_train.info()

We can see that the dataset contains 1 numerical and 10 categorical features.

Let's also calculate the percentage of NaNs for each column of the train and test set:

In [ ]:
df_train.isnull().mean() * 100

In [ ]:
df_test.isnull().mean() * 100

We can see that we have some NANs we have to take care of later. In addition, let's check for duplicated rows in train and test set:

In [ ]:
df_train.duplicated().sum()

In [ ]:
df_test.duplicated().sum()

There are not duplicates in the datasets which is good :). Let's also check how many identical rows appear in both train and test set:

In [ ]:
len(pd.merge(df_train.drop(columns=['price']), df_test))

Ok, that's interesting and probably results from having added the original dataset to the training data. Let's summarize our first insights:

####  **First insights into the dataset:**

* We have missing values which we have to impute
* We have categorical data which we have to encode
* Finally, we also have features (e.g. 'engine') which contain a lot of potential for feature engineering

## **EDA (Exploratory Data Analysis)**

Let's now do an EDA to deepen our understanding of the dataset. We will use the Auto EDA tool ydata_profiling for this purpose:

In [ ]:
profile = ProfileReport(df_train, title="Profiling Report")

In [ ]:
profile.to_notebook_iframe()

#### **Some additional insights from the EDA**:

* 'clean_title' is either 'True' or was not reported
* We have high correlations between some features
* 'fuel_type' is highly imbalanced
* 'price' is highly skewed 


## **Feature Engineering & Preprocessing**

We now will engineer some useful features for our regression task and fill the NANs. The following code cell will build up several new features (e.g. the number of cylinders) and will fill the NANs and was gratefully taken from https://www.kaggle.com/code/ravi20076/playgrous4e09-baseline-v1:

In [ ]:
# thanks to https://www.kaggle.com/code/ravi20076/playgrous4e09-baseline-v1 for the following code
class FtreMaker:
    "This class makes new features using public discussions and kernel ideas"
    
    def __init__(self):
        self.fuel_type_dict = {
            'Gasoline': 0,
            'Hybrid': 1,
            'E85 Flex Fuel': 2,
            'uknown': 3,
            'Diesel': 4,
            'dash': 5,
            'Plug-In Hybrid': 6,
            'not supported': 7
        }

        self.accident_dict = {
            'None reported': 0,
            'At least 1 accident or damage reported': 1,
            'uknown': 2
        }

        self.clean_title_dict = {
            'Yes': 0,
            'uknown': 1
        }

        self.expensive_ext_color = \
        [  'Blue Caelum', 'Dark Sapphire', 'Bianco Monocerus', 'C / C', 'Ice',
           'Tempest', 'Beluga Black', 'Bianco Icarus Metallic', 
           'BLU ELEOS', 'Shadow Black', 'Nero Noctis', 'Sandstone Metallic',
           'Lizard Green', 'Balloon White', 'Onyx', 'Donington Grey Metallic',
           'China Blue', 'Diamond White', 'Rosso Corsa', 'Granite',
           'Rosso Mars Metallic', 'Carpathian Grey', 'Kemora Gray Metallic',
           'Grigio Nimbus', 'dash', 'Bianco Isis', 'Python Green', 'Fountain Blue',
           'Custom Color', 'Vega Blue', 'Designo Magno Matte',
           'Brands Hatch Gray Metallic', 'Rift Metallic', 'Gentian Blue Metallic',
           'Arancio Borealis', 'BLUE', 'Aventurine Green Metallic', 'Apex Blue',
           'Daytona Gray Pearl Effect', 'Daytona Gray Pearl Effect w/ Black Roof',
           'Matte White', 'Carpathian Grey Premium Metallic', 'Blue Metallic',
           'Santorini Black Metallic', 'Quartzite Grey Metallic',
           'Carrara White Metallic', 'BLACK', 'Kinetic Blue', 'Nero Daytona'
        ]

        self.expensive_int_color = \
        ['Dark Auburn', 'Hotspur', 'Cobalt Blue', 'Beluga Hide', 'Linen',
         'Beluga', 'Black / Brown', 'Nero Ade', 'Sahara Tan', 'Portland']

        self.expensive_hp = \
        [443.0, 473.0, 493.0, 502.0, 521.0, 542.0, 
         543.0, 571.0, 572.0, 573.0, 580.0,
         591.0, 602.0, 611.0, 616.0, 620.0, 624.0, 
         640.0, 641.0, 651.0, 710.0, 715.0, 760.0, 788.0, 797.0
        ]
        
        self.series_pattern = \
        re.compile(r'^[A-Za-z0-9\-]+')
        
        self.version_pattern = \
        re.compile(r'([0-9]+\.[0-9]+[A-Za-z]*)|([A-Z]+[0-9]*)')
        
        self.trim_pattern = \
        re.compile(r'\b(Base|Sport|Premium|Ultimate|XLT|LZ|LT|Plus|Touring|SE|LE|Limited|Platinum|Performance|S|V6|GT|EX|SX|XLE|SR|SL|SV|XSE|TRD|RS|GranSport|Signature|Quad Cab|DRW|Cabriolet|Carbon Edition|Trail Boss|Prestige|Essence|Reserve|xDrive|4MATIC|PreRunner|EcoBoost|Scat Pack|Competition|Adventure Package|Laramie|Grand Touring|Long Range)\b')
        
        self.hp_pattern = r'(\d+(\.\d+)?)HP'
        self.displacement_pattern = r'(\d+\.\d+)L'
        self.cylinder_pattern = r'(\d+) Cylinder'

        self.engine_configs = {
            'V6': 'V6', 
            'V8': 'V8', 
            'V10': 'V10', 
            'V12': 'V12',
            'Straight 6': 'Straight 6', 
            'Flat 6': 'Flat 6', 
            'I4': 'I4'
        }

        self.forced_induction = {'Turbo', 'Twin Turbo'}
        self.valve_configs    = {'DOHC', 'SOHC', 'GDI', 'PDI'}
        self.fuel_systems     = {'MPFI', 'GDI', 'PDI', 'TFSI'}
        
    def fit(self, X: pd.DataFrame, y = None, **params):
        return self
    
    def _make_mdlparts(self, model: str):
        "This method extracts model components"
        
        series  = self.series_pattern.search(model)
        version = self.version_pattern.search(model)
        trim    = self.trim_pattern.search(model)

        return {
            'Series'  : series.group(0) if series else "uknown",
            'Version' : version.group(0) if version else "uknown",
            'Trim'    : trim.group(0) if trim else "uknown"
        }
    
    def _parse_engine(self, engine_desc):
        "This metod processes the engine description feature and returns sub-parts"
        
        int_default_na = -1
        obj_default_na = "uknown"
        
        features = {
            'horsepower': int_default_na, 
            'displacement': int_default_na, 
            'cylinder_count': int_default_na,
            'engine_configuration':obj_default_na, 
            'forced_induction': obj_default_na,
            'valve_configuration': obj_default_na, 
            'fuel_system': obj_default_na, 
            'mild_hybrid': False,
        }
        
        patterns = {
            'horsepower'    : (self.hp_pattern, lambda x: float(x)),
            'displacement'  : (self.displacement_pattern, lambda x: float(x)),
            'cylinder_count': (self.cylinder_pattern, lambda x: int(x)),
        }
        
        for feature, (pattern, convert) in patterns.items():
            match = re.search(pattern, engine_desc)
            if match:
                try:
                    features[feature] = convert(match.group(1))
                except ValueError:
                    pass  
        
        features['engine_configuration'] = \
        next((config for config, name in self.engine_configs.items() if config in engine_desc), 
             obj_default_na
            )
        
        features['forced_induction'] = \
        next((induction for induction in self.forced_induction if induction in engine_desc),
             obj_default_na
            )
        
        features['valve_configuration'] = \
        next((valve for valve in self.valve_configs if valve in engine_desc), 
             obj_default_na
            )
        
        features['fuel_system'] = \
        next((system for system in self.fuel_systems if system in engine_desc), 
             obj_default_na
            )
        
        features['mild_hybrid'] = 'Mild Electric Hybrid' in engine_desc
        
        return features
    
    def _get_gear_number(self, trans):
        match = re.search(r'(\d+)[-\s]?Speed', trans)
        return int(match.group(1)) if match else -1

    def _get_transmission_type(self, trans):
        if 'A/T' in trans or 'Automatic' in trans:
            return 'Automatic'
        elif 'M/T' in trans or 'Manual' in trans:
            return 'Manual'
        elif 'CVT' in trans:
            return 'CVT'
        elif 'DCT' in trans:
            return 'DCT'
        else:
            return 'Other'

    def _get_special_features(self, trans):
        features = []
        if 'Dual Shift Mode' in trans:
            features.append('Dual Shift Mode')
        if 'Auto-Shift' in trans:
            features.append('Auto-Shift')
        if 'Overdrive' in trans:
            features.append('Overdrive')
        if 'Electronically Controlled' in trans:
            features.append('Electronically Controlled')
        if 'Variable' in trans:
            features.append('Variable')
        return ', '.join(features) if features else "uknown"

    def _get_transmission_dsg(self, trans):
        if 'A/T' in trans:
            return 'A/T'
        elif 'M/T' in trans:
            return 'M/T'
        elif 'CVT' in trans:
            return 'CVT'
        elif 'DCT' in trans:
            return 'DCT'
        else:
            return "uknown"

    def transform(self, X: pd.DataFrame, y = None, **params):
        "This method makes new features from the existing dataset"

        df = X.copy()
        print(f"\n\n---> Starting shape = {df.shape}")
        
        print(f"---> Imputing category columns")
        cat_ftre     = df.select_dtypes("object").columns
        df[cat_ftre] = df[cat_ftre].fillna("uknown")
        
        print(f"---> Performing -150 feature transforms")
        df['fuel_type_encoded']   = df['fuel_type'].map(self.fuel_type_dict)
        df['accident_encoded']    = df['accident'].map(self.accident_dict)
        df['clean_title_encoded'] = df['clean_title'].map(self.clean_title_dict)
        
        df['expensive_color_ext_encoded'] = \
        df.ext_col.isin(self.expensive_ext_color).astype(int)
        
        df['expensive_color_int_encoded'] = \
        df.int_col.isin(self.expensive_int_color).astype(int)
        
        df['expensive_hp'] = \
        df['engine'].str.extract(r'(\d+\.?\d*)HP').\
        astype(float).isin(self.expensive_hp).\
        astype(int)
        
        df['cylinder'] = \
        df['engine'].str.extract(r'(\d+\.?\d*) Cylinder').astype(float) 
        
        df['got_V'] = df['model'].str.extract(r'(\d+\.?\d*) V').notna().astype(int)
        
        print(f"---> Making car model components")
        mdlparts = df['model'].apply(self._make_mdlparts).apply(pd.Series)
        df       = pd.concat([df, mdlparts], axis=1)
        del mdlparts
        
        print(f"---> Making engine components")
        unique_engines  = df['engine'].unique()
        engine_features = [self._parse_engine(engine) for engine in unique_engines]
        engine_features = pd.DataFrame(engine_features, index = unique_engines).reset_index()
        engine_features.rename(columns={'index': 'engine'}, inplace=True)
        
        df = df.merge(engine_features, how = "left", on = "engine")
        del engine_features,unique_engines
 
        print(f"---> Making transmission components")
        df['nb_gears']   = \
        df['transmission'].apply(self._get_gear_number)
        df['transmission_mode'] = \
        df['transmission'].apply(self._get_transmission_type)
        df['transmission_special']  = \
        df['transmission'].apply(self._get_special_features)
        df['transmission_dsg'] = \
        df['transmission'].apply(self._get_transmission_dsg)
        
        print(f"---> Imputing all numeric and categorical features at the end")
        num_ftre = df.select_dtypes(include=[np.number]).columns
        cat_ftre = df.select_dtypes(include=['object']).columns
        
        df[num_ftre] = df[num_ftre].fillna(-1)
        df[cat_ftre] = df[cat_ftre].fillna("uknown").astype("category")
        
        #df["Source"] = df["Source"].astype(str)
  
        print(f"\n---> Final shape = {df.shape}")
        return df
    
xform = \
Pipeline(steps = [("MakeFtre", FtreMaker())],verbose = True)
X_train = df_train.drop(columns=['price'])
y_train = df_train['price']
X_test  = df_test.copy()

df_train = xform.fit_transform(X_train, y_train)
df_test  = xform.transform(X_test)
df_train['price'] = y_train

Now, we use target encoding to encode all features. For this, we first define all categorical features and then execute the encoding:

In [ ]:
cat_features = ['brand', 'model', 'model_year', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
       'fuel_type_encoded', 'accident_encoded', 'clean_title_encoded',
       'expensive_color_ext_encoded', 'expensive_color_int_encoded',
       'expensive_hp', 'cylinder', 'got_V', 'Series', 'Version', 'Trim',
       'horsepower', 'displacement', 'cylinder_count', 'engine_configuration',
       'forced_induction', 'valve_configuration', 'fuel_system', 'mild_hybrid',
       'nb_gears', 'transmission_mode', 'transmission_special',
       'transmission_dsg']

# we also save the column names for later before encoding the features
col_names = df_train.columns

In [ ]:
tar_encoder = TargetEncoder(smooth="auto", target_type='continuous', cv=5, random_state=0)

tar_cols_train = pd.DataFrame(tar_encoder.fit_transform(df_train[cat_features], df_train['price']))
tar_cols_test = pd.DataFrame(tar_encoder.transform(df_test[cat_features]))

# One-hot encoding removes index, we need to put it back
tar_cols_train.index = df_train.index
tar_cols_test.index = df_test.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = df_train.drop(cat_features, axis=1)
num_X_test= df_test.drop(cat_features, axis=1)

# Add one-hot encoded columns to numerical features
df_train = pd.concat([num_X_train, tar_cols_train], axis=1)
df_test = pd.concat([num_X_test, tar_cols_test], axis=1)

# Ensure all columns have string type
df_train.columns = df_train.columns.astype(str)
df_test.columns = df_test.columns.astype(str)

Let's now analyze the most important features for the regression task. A method for this is to use a shallow regression tree (inspired by this https://www.kaggle.com/competitions/playground-series-s4e6/discussion/509073) and we can see that the 'model' feature seems to be the most important feature since it is used in the first levels of the tree:

In [ ]:
# thanks to https://www.kaggle.com/code/ambrosm/pss4e6-eda-which-makes-sense for (the most parts of) the code
dt = DecisionTreeRegressor(max_depth=3)
dt.fit(df_train, df_train.price)

plt.figure(figsize=(16, 6))
plot_tree(dt, feature_names=col_names, class_names=['price'], fontsize=7, impurity=False, filled=True, ax=plt.gca())
plt.show()

## **Modeling**

We now train our model via Auto ML with the help of the package 'Autogluon'. It is as simple as follows:

In [ ]:
model = TabularPredictor(label='price',
                            eval_metric='rmse',
                            problem_type='regression').fit(df_train,
                                                       presets='best_quality',
                                                       time_limit=3600,
                                                       excluded_model_types=['KNN'])

In [ ]:
model.leaderboard()

We can see that the best model performs well in the validation and increasing the training time can significantly improve the scores further (especially on the leaderboard).

We can use the best performing model for producing the submission predictions as follows:

In [ ]:
y_pred = model.predict(df_test)
df_sub['price'] = y_pred
df_sub.to_csv('submission.csv', index=False)

In [ ]:
df_sub_test = pd.read_csv('submission.csv')
df_sub_test.head()

## **Summary and possible next steps:**
In summary, we successfully built an EDA and a model for the PS4E9 task using Auto EDA and Auto ML. Based on this, next steps could be:
* Engineer additonal useful features
* Increase model complexity e.g. through longer Auto ML training or by combining the Auto ML model with other models in an ensemble